# Notebook with Minimal example to run hposuite

In [1]:
from lib.benchmarks import BENCHMARKS
from lib.optimizers import OPTIMIZERS

print(BENCHMARKS)
print(OPTIMIZERS)

{'Ex_Functional_Bench': <hpoglue.benchmark.FunctionalBenchmark object at 0x6fff6c16e2c0>}
{'RandomSearch': <class 'lib.optimizers.random_search.RandomSearch'>, 'Multifidelity_Opt_Example': <class 'lib.optimizers.ex_multifidelity.Ex_Multifidelity'>}


## Running Random Search on a Functional Benchmark (eg: ACKLEY)

In [ ]:
func_benchmark = BENCHMARKS["Ex_Functional_Bench"]
rs = OPTIMIZERS["RandomSearch"]

In [ ]:
from hposuite import create_study

study = create_study(
    name="hposuite_demo",
    output_dir="../../hposuite-output",
    optimizers=rs,
    benchmarks=func_benchmark,
    num_seeds=5,
    budget=100,
)

study.optimize(overwrite=True)

## Running a MultiFidelity Optimizer (eg: Hyperband) on a Surrogate Benchmark (eg: PD1 Cifar100 Wide Resnet 2048)

In [ ]:
sg_benchmark = BENCHMARKS["Ex_Surrogate_Bench"]
mf_optimizer = OPTIMIZERS["Ex_Multifidelity_Opt"]

In [ ]:
from hposuite import create_study

study = create_study(
    name="hposuite_demo",
    output_dir="../../hposuite-output",
    optimizers=mf_optimizer,
    benchmarks=sg_benchmark,
    num_seeds=5,
    budget=100,
)

study.optimize(overwrite=True)